In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd

from nba_api.stats.endpoints import teamgamelogs
from nba_api.stats.static import teams
nba_teams = teams.get_teams()
teamIDs = [team['id'] for team in nba_teams]

In [2]:
games = pd.read_csv(r'.\data\nba_games_all.csv', index_col=0)
ou = pd.read_csv(r'.\data\nba_betting_totals.csv', index_col=0)
spread = pd.read_csv(r'.\data\nba_betting_spread.csv', index_col=0)
ml = pd.read_csv(r'.\data\nba_betting_money_line.csv', index_col=0)

In [3]:
ouSpread = ou.merge(spread, on=['game_id','book_name','book_id','team_id','a_team_id'],suffixes=('_ou', '_spread'))
bets = ouSpread.merge(ml, on=['game_id','book_name','book_id','team_id','a_team_id'])

In [4]:
bets.head()

,book_name,book_id,team_id,a_team_id,total1,total2,price1_ou,price2_ou,spread1,spread2,price1_spread,price2_spread,price1,price2
game_id,,,,,,,,,,,,,,
21100131,Pinnacle Sports,238,1610612740,1610612743,192.0,192.0,-105.0,-105.0,13.5,-13.5,-105.0,-105.0,915.0,-1250.0
21100131,5Dimes,19,1610612740,1610612743,192.0,192.0,-105.0,-105.0,13.5,-13.5,-105.0,-105.0,950.0,-1150.0
21100131,Bookmaker,93,1610612740,1610612743,192.0,192.0,-110.0,-110.0,14.0,-14.0,-110.0,-110.0,900.0,-1400.0
21100131,BetOnline,1096,1610612740,1610612743,191.5,191.5,-110.0,-110.0,13.5,-13.5,-105.0,-115.0,850.0,-1200.0
21100131,Heritage,169,1610612740,1610612743,191.5,191.5,-110.0,-110.0,13.5,-13.5,-110.0,-110.0,890.0,-1340.0


In [5]:
years = ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019']

games = games.dropna()
games[games['game_date'].str.contains('|'.join(years))].head()
#games = games[(games['game_date'] > '1/1/2010') & (games['game_date'] < '12/31/2020')]

,game_date,matchup,team_id,is_home,wl,w,l,w_pct,min,fgm,...,ast,stl,blk,tov,pf,pts,a_team_id,season_year,season_type,season
game_id,,,,,,,,,,,,,,,,,,,,,
20900965,2010-03-12,PHI vs. CLE,1610612739,f,W,51.0,15.0,0.773,240,38.0,...,26.0,6.0,3.0,11.0,16.0,100,1610612755,2009,Regular Season,2009-10
20900940,2010-03-08,SAS @ CLE,1610612739,t,W,50.0,15.0,0.769,240,36.0,...,21.0,8.0,2.0,12.0,17.0,97,1610612759,2009,Regular Season,2009-10
20900857,2010-02-25,BOS vs. CLE,1610612739,f,W,45.0,14.0,0.763,240,41.0,...,24.0,5.0,6.0,11.0,19.0,108,1610612738,2009,Regular Season,2009-10
20900638,2010-01-23,OKC @ CLE,1610612739,t,W,34.0,11.0,0.756,240,34.0,...,19.0,3.0,6.0,17.0,25.0,100,1610612760,2009,Regular Season,2009-10
20900477,2010-01-02,NJN vs. CLE,1610612739,f,W,27.0,8.0,0.771,240,32.0,...,19.0,4.0,5.0,12.0,24.0,94,1610612751,2009,Regular Season,2009-10


In [6]:
bookies = bets['book_name'].unique()
byBook = []

bets = bets.rename(columns={'price1':'price1_ml', 'price2':'price2_ml'})
print(bookies)

for i in range(len(bookies)):
    byBook.append(bets[bets['book_name'] == bookies[i]])
    byBook[i] = byBook[i].rename(columns={'total1':'total1_'+bookies[i], 
                                          'total2':'total2_'+bookies[i], 
                                          'price1_ou':'price1_ou_'+bookies[i], 
                                          'price2_ou':'price2_ou_'+bookies[i], 
                                          'spread1':'spread1_'+bookies[i], 
                                          'spread2':'spread2_'+bookies[i],
                                          'price1_spread':'price1_spread_'+bookies[i],
                                          'price2_spread':'price2_spread_'+bookies[i],
                                          'price1_ml':'price1_ml_'+bookies[i],
                                          'price2_ml':'price2_ml_'+bookies[i]})
    byBook[i] = byBook[i].drop(columns=['book_name','book_id','team_id','a_team_id'])
byBook[1].head()
bets = byBook[0]
for i in range(1,len(byBook)):
    bets = bets.merge(byBook[i], on="game_id")
bets.head()

['Pinnacle Sports' '5Dimes' 'Bookmaker' 'BetOnline' 'Heritage' 'Intertops'
 'YouWager' 'JustBet' 'Sportsbetting' 'Bovada']


,total1_Pinnacle Sports,total2_Pinnacle Sports,price1_ou_Pinnacle Sports,price2_ou_Pinnacle Sports,spread1_Pinnacle Sports,spread2_Pinnacle Sports,price1_spread_Pinnacle Sports,price2_spread_Pinnacle Sports,price1_ml_Pinnacle Sports,price2_ml_Pinnacle Sports,...,total1_Bovada,total2_Bovada,price1_ou_Bovada,price2_ou_Bovada,spread1_Bovada,spread2_Bovada,price1_spread_Bovada,price2_spread_Bovada,price1_ml_Bovada,price2_ml_Bovada
game_id,,,,,,,,,,,,,,,,,,,,,
21100137,193.5,193.5,-105.0,-105.0,-2.5,2.5,-105.0,-105.0,-130.0,118.0,...,193.0,193.0,-110.0,-110.0,-3.0,3.0,-110.0,-110.0,-150.0,130.0
21100148,183.5,183.5,-105.0,-105.0,-6.5,6.5,-105.0,-105.0,-255.0,227.0,...,184.0,184.0,-105.0,-115.0,-7.0,7.0,-105.0,-115.0,-300.0,250.0
21100149,198.5,198.5,-105.0,-105.0,7.0,-7.0,-105.0,-105.0,244.0,-275.0,...,199.0,199.0,-110.0,-110.0,7.0,-7.0,-110.0,-110.0,250.0,-300.0
21100152,186.0,186.0,-107.0,-103.0,3.0,-3.0,-104.0,-106.0,133.0,-147.0,...,185.0,185.0,-115.0,-105.0,3.5,-3.5,-115.0,-105.0,135.0,-155.0
21100153,203.5,203.5,-107.0,-103.0,-3.0,3.0,-104.0,-106.0,-150.0,136.0,...,204.0,204.0,-110.0,-110.0,-3.5,3.5,-115.0,-105.0,-165.0,145.0


In [7]:
games = games.merge(bets, on="game_id")
games.head()

,game_date,matchup,team_id,is_home,wl,w,l,w_pct,min,fgm,...,total1_Bovada,total2_Bovada,price1_ou_Bovada,price2_ou_Bovada,spread1_Bovada,spread2_Bovada,price1_spread_Bovada,price2_spread_Bovada,price1_ml_Bovada,price2_ml_Bovada
game_id,,,,,,,,,,,,,,,,,,,,,
21600745,2017-02-02,GSW @ LAC,1610612746,t,L,31.0,19.0,0.620,240,40.0,...,230.0,230.0,-110.0,-110.0,-8.5,8.5,-110.0,-110.0,-360.0,280.0
21600745,2017-02-02,LAC vs. GSW,1610612744,f,W,43.0,7.0,0.860,240,52.0,...,230.0,230.0,-110.0,-110.0,-8.5,8.5,-110.0,-110.0,-360.0,280.0
21600739,2017-02-01,PHX vs. LAC,1610612746,f,W,31.0,18.0,0.633,240,44.0,...,220.0,220.0,-115.0,-105.0,-3.0,3.0,-105.0,-115.0,-145.0,125.0
21600739,2017-02-01,LAC @ PHX,1610612756,t,L,15.0,34.0,0.306,240,39.0,...,220.0,220.0,-115.0,-105.0,-3.0,3.0,-105.0,-115.0,-145.0,125.0
21600675,2017-01-24,PHI vs. LAC,1610612746,f,L,30.0,17.0,0.638,240,33.0,...,204.0,204.0,-115.0,-105.0,-5.0,5.0,-110.0,-110.0,-200.0,170.0


In [8]:
games.to_csv(r'.\data\betting.csv')